In [3]:
import sys
sys.path.append('../src')

from wikidataDB import Session, WikidataID, WikidataEntity
from wikidataEmbed import WikidataTextifier, JinaAIEmbeddings
from tqdm import tqdm
import json
import os
import torch
from langchain_core.documents import Document
from ragstack_langchain.graph_store import CassandraGraphStore
import cassio
from langchain_astradb import AstraDBVectorStore, AstraDBGraphVectorStore
from astrapy.info import CollectionVectorServiceOptions
from astrapy import DataAPIClient
from transformers import AutoTokenizer
import requests
import time
import pickle

In [23]:
keyspace

'default_keyspace'

In [ ]:
import os
import cassio
from cassio.config import check_resolve_keyspace, check_resolve_session

cassio.init(
    token=datastax_token["ASTRA_DB_APPLICATION_TOKEN"],
    database_id=datastax_token["ASTRA_DB_DATABASE_ID"],
    keyspace=datastax_token["ASTRA_DB_KEYSPACE"]
)

session = check_resolve_session()
keyspace = check_resolve_keyspace()
table = "default_keyspace.wikidata_test_v1"
page_size = 1000
last_token = None

with tqdm(total=530994) as progressbar:
    while True:
        # Construct query
        if last_token is None:
            query = f"SELECT key, token(key) FROM {table} LIMIT {page_size};"
        else:
            query = f"SELECT key, token(key) FROM {table} WHERE token(key) > {last_token} LIMIT {page_size};"

        # Execute query
        rows = session.execute(query)
        rows_fetched = 0

        # Process rows
        for row in rows:
            qid = row.key[1].split("_")[0]
            sample_ids.loc[sample_ids['QID'] == qid, 'in_wikidata_test_v1'] = True
            last_token = row.system_token_key  # Update the last token
            rows_fetched += 1
            progressbar.update(1)

        # Break if no more rows are fetched
        if rows_fetched < page_size:
            break

  1%|          | 3857/530994 [00:51<1:52:38, 77.99it/s]

In [16]:
datastax_token = json.load(open("../API_tokens/datastax_wikidata_nvidia.json"))
COLLECTION_NAME = 'wikidata_test_v1'

client = DataAPIClient(datastax_token['ASTRA_DB_APPLICATION_TOKEN'])
database0 = client.get_database(datastax_token['ASTRA_DB_API_ENDPOINT'])
wikiDataCollection = database0.get_collection(COLLECTION_NAME)

sample_ids = pickle.load(open("../data/Evaluation Data/Sample IDs (EN).pkl", "rb"))
sample_ids['in_wikidata_test_v1'] = False

for qid in tqdm((sample_ids[~sample_ids['in_wikidata_test_v1']]['QID'].values)):
  item = wikiDataCollection.find_one({"_id": qid})
  if item is not None:
    sample_ids.loc[sample_ids['QID'] == qid, 'in_wikidata_test_v1'] = True

  1%|          | 3824/422484 [10:34<19:17:41,  6.03it/s]


KeyboardInterrupt: 

In [ ]:
datastax_token = json.load(open("../API_tokens/datastax_wikidata_nvidia.json"))
os.environ["ASTRA_DB_DATABASE_ID"] = datastax_token['database_id']
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = datastax_token['token']

BATCH_SIZE = 1
BATCH_LENGTH = 5000

cassio.init(auto=True)
embeddings = JinaAIEmbeddings(embedding_dim=1024)
textifier = WikidataTextifier(with_claim_aliases=True, with_property_aliases=False)
graph_store = CassandraGraphStore(
    embeddings,
    node_table="Wikidata_entities_v1",
)

In [ ]:
with tqdm(total=9203531) as progressbar:
    with Session() as session:
        entities = session.query(WikidataEntity).join(WikidataID, WikidataEntity.id == WikidataID.id).filter(WikidataID.in_wikipedia == True).offset(offset).yield_per(1000)
        doc_batch = []
        batch_length = 0
        progressbar.update(offset)

        for entity in entities:
            progressbar.update(1)
            chunks = embeddings.chunk_text(entity, textifier)
            for chunk_i in range(len(chunks)):
                doc = Document(page_content=chunks[chunk_i], metadata={"QID": entity.id, "ChunkID": chunk_i+1})
                if f"{entity.id}_{chunk_i+1}" not in prev_data:
                    doc_batch.append(doc)
                    batch_length += len(chunks[chunk_i])

                if batch_length >= BATCH_LENGTH:
                    try:
                        graph_store.add_documents(doc_batch)
                        torch.cuda.empty_cache()
                        with open("../data/Wikidata/pushed_embeddings.json", "a+") as file:
                            file.write(", ".join([f"\"{d.metadata["QID"]}_{d.metadata["ChunkID"]}\": 1" for d in doc_batch]) +", ")

                        progressbar.set_description(f"Batch Size: {len(doc_batch)}")
                        doc_batch = []
                        batch_length = 0
                    except Exception as e:
                        torch.cuda.empty_cache()
                        progressbar.set_description(f"Batch Size: 1")
                        print(e)

                        while len(doc_batch) > 0:
                            doc = doc_batch.pop()
                            embeddings.model = embeddings.model.to('cpu')
                            graph_store.add_documents([doc])
                            torch.cuda.empty_cache()
                            with open("../data/Wikidata/pushed_embeddings.json", "a+") as file:
                                file.write(f"\"{doc.metadata["QID"]}_{doc.metadata["ChunkID"]}\": 1, ")
                        batch_length = 0
                        embeddings.model = embeddings.model.to('cuda')

In [6]:
cassio.init(auto=True)
from cassio.config import check_resolve_keyspace, check_resolve_session

session = check_resolve_session()
keyspace = check_resolve_keyspace()

In [ ]:
session.execute(f"DROP TABLE IF EXISTS {keyspace}.wikidata_entities")

In [12]:
datastax_token = json.load(open("../API_tokens/datastax_wikidata_nvidia.json"))
ASTRA_DB_DATABASE_ID = datastax_token['ASTRA_DB_DATABASE_ID']
ASTRA_DB_APPLICATION_TOKEN = datastax_token['ASTRA_DB_APPLICATION_TOKEN']
ASTRA_DB_API_ENDPOINT = datastax_token["ASTRA_DB_API_ENDPOINT"]
ASTRA_DB_KEYSPACE = datastax_token["ASTRA_DB_KEYSPACE"]

BATCH_SIZE = 100

textifier = WikidataTextifier(with_claim_aliases=False, with_property_aliases=False)
tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-large-unsupervised', trust_remote_code=True, clean_up_tokenization_spaces=False)

collection_vector_service_options = CollectionVectorServiceOptions(
    provider="nvidia",
    model_name="NV-Embed-QA"
)

graph_store = AstraDBVectorStore(
    collection_name="wikidata",
    collection_vector_service_options=collection_vector_service_options,
    token=ASTRA_DB_APPLICATION_TOKEN,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    namespace=ASTRA_DB_KEYSPACE,
)

vector_retriever = graph_store.as_retriever(search_kwargs={"k": 1000, "depth": 0})
results = vector_retriever.get_relevant_documents("Is this a question?")

In [5]:
import cassio
from cassio.config import check_resolve_keyspace, check_resolve_session

cassio.init(database_id=ASTRA_DB_DATABASE_ID, token=ASTRA_DB_APPLICATION_TOKEN)
session_cassio = check_resolve_session()
keyspace = check_resolve_keyspace()

In [ ]:
offset = 8342800
with tqdm(total=9203531) as progressbar:
    with Session() as session:
        entities = session.query(WikidataEntity).join(WikidataID, WikidataEntity.id == WikidataID.id).filter(WikidataID.in_wikipedia == True).offset(offset).yield_per(BATCH_SIZE)
        progressbar.update(offset)
        n = 0
        for entity in entities:
            progressbar.update(1)
            results = session_cassio.execute(f"SELECT * FROM {ASTRA_DB_KEYSPACE}.wikidata WHERE key = (1, '{str(entity.id)}_1') LIMIT 1;")
            n += 1
            if not results.one():
                break

In [ ]:
offset = 8342800
with tqdm(total=9203531) as progressbar:
    with Session() as session:
        entities = session.query(WikidataEntity).join(WikidataID, WikidataEntity.id == WikidataID.id).filter(WikidataID.in_wikipedia == True).offset(offset).yield_per(BATCH_SIZE)
        progressbar.update(offset)
        doc_batch = []
        ids_batch = []

        for entity in entities:
            progressbar.update(1)
            chunks = textifier.chunk_text(entity, tokenizer)
            for chunk_i in range(len(chunks)):
                doc = Document(page_content=chunks[chunk_i], metadata={"QID": entity.id, "ChunkID": chunk_i+1})
                doc_batch.append(doc)
                ids_batch.append(f"{entity.id}_{chunk_i+1}")

                if len(doc_batch) >= BATCH_SIZE:
                    try:
                        graph_store.add_documents(doc_batch, ids=ids_batch)
                        doc_batch = []
                        ids_batch = []
                    except Exception as e:
                        print(e)
                        while True:
                            try:
                                response = requests.get("https://www.google.com", timeout=5)
                                if response.status_code == 200:
                                    break
                            except Exception as e:
                                print("Waiting for internet connection...")
                                time.sleep(5)

        if len(doc_batch) > 0:
            graph_store.add_documents(doc_batch, ids=ids_batch)

In [2]:
datastax_token = json.load(open("../API_tokens/datastax_wikidata_nvidia.json"))
ASTRA_DB_DATABASE_ID = datastax_token['ASTRA_DB_DATABASE_ID']
ASTRA_DB_APPLICATION_TOKEN = datastax_token['ASTRA_DB_APPLICATION_TOKEN']
ASTRA_DB_API_ENDPOINT = datastax_token["ASTRA_DB_API_ENDPOINT"]
ASTRA_DB_KEYSPACE = datastax_token["ASTRA_DB_KEYSPACE"]

BATCH_SIZE = 1

textifier = WikidataTextifier(with_claim_aliases=False, with_property_aliases=False)
embeddings = JinaAIEmbeddings(embedding_dim=1024)

graph_store = AstraDBVectorStore(
    collection_name="wikidata_test_v1",
    embedding=embeddings,
    token=ASTRA_DB_APPLICATION_TOKEN,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    namespace=ASTRA_DB_KEYSPACE,
)

flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn i

In [5]:
offset = 8848000
with tqdm(total=9203531) as progressbar:
    with Session() as session:
        entities = session.query(WikidataEntity).join(WikidataID, WikidataEntity.id == WikidataID.id).filter(WikidataID.in_wikipedia == True).offset(offset).yield_per(1000)
        progressbar.update(offset)
        doc_batch = []
        ids_batch = []

        for entity in entities:
            progressbar.update(1)
            if entity.id in sample_ids['QID'].values:
                chunks = textifier.chunk_text(entity, embeddings.tokenizer, max_length=1024)
                for chunk_i in range(len(chunks)):
                    doc = Document(page_content=chunks[chunk_i], metadata={"QID": entity.id, "ChunkID": chunk_i+1})
                    doc_batch.append(doc)
                    ids_batch.append(f"{entity.id}_{chunk_i+1}")

                    if len(doc_batch) >= BATCH_SIZE:
                        try:
                            graph_store.add_documents(doc_batch, ids=ids_batch)
                            torch.cuda.empty_cache()
                            doc_batch = []
                            ids_batch = []
                        except Exception as e:
                            print(e)
                            while True:
                                try:
                                    response = requests.get("https://www.google.com", timeout=5)
                                    if response.status_code == 200:
                                        break
                                except Exception as e:
                                    print("Waiting for internet connection...")
                                    time.sleep(5)

        if len(doc_batch) > 0:
            graph_store.add_documents(doc_batch, ids=ids_batch)

  0%|          | 0/9203531 [00:00<?, ?it/s]

 96%|█████████▌| 8848001/9203531 [00:11<00:00, 793069.15it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (21632 > 8194). Running this sequence through the model will result in indexing errors
9203786it [54:56, 2791.98it/s]                               


In [2]:
datastax_token = json.load(open("../API_tokens/datastax_wikidata_nvidia.json"))
ASTRA_DB_DATABASE_ID = datastax_token['ASTRA_DB_DATABASE_ID']
ASTRA_DB_APPLICATION_TOKEN = datastax_token['ASTRA_DB_APPLICATION_TOKEN']
ASTRA_DB_API_ENDPOINT = datastax_token["ASTRA_DB_API_ENDPOINT"]
ASTRA_DB_KEYSPACE = datastax_token["ASTRA_DB_KEYSPACE"]

BATCH_SIZE = 100

textifier = WikidataTextifier(with_claim_aliases=False, with_property_aliases=False)
tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-large-unsupervised', trust_remote_code=True, clean_up_tokenization_spaces=False)

collection_vector_service_options = CollectionVectorServiceOptions(
    provider="nvidia",
    model_name="NV-Embed-QA"
)

graph_store = AstraDBVectorStore(
    collection_name="wikidata_test_v2",
    collection_vector_service_options=collection_vector_service_options,
    token=ASTRA_DB_APPLICATION_TOKEN,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    namespace=ASTRA_DB_KEYSPACE,
)

In [9]:
from astrapy import DataAPIClient

client = DataAPIClient(ASTRA_DB_APPLICATION_TOKEN)
database = client.get_database(ASTRA_DB_API_ENDPOINT)
wikidataCollection = database.get_collection("wikidata_test_v1")

In [11]:
doc_iterator = wikidataCollection.find()
for doc in doc_iterator:
    print(doc['metadata']['QID'])
    break

Q855252


In [15]:
from wikidataDB import Session, WikidataID, WikidataEntity
from wikidataEmbed import WikidataTextifier, JinaAIEmbeddings

import json
from langchain_astradb import AstraDBVectorStore
from langchain_core.documents import Document
from astrapy.info import CollectionVectorServiceOptions
from transformers import AutoTokenizer
from tqdm import tqdm
from langchain_core.documents import Document
from astrapy import DataAPIClient
import requests
import time
import os
import pickle
import torch
from datetime import datetime
import hashlib  # Import the hashlib library
import json

NVIDIA = "true"
#SAMPLE = os.getenv("SAMPLE", "false").lower() == "true"
SAMPLE=False
BATCH_SIZE = 900
OFFSET = int(os.getenv("OFFSET", 0))
API_KEY_FILENAME = os.getenv("API_KEY", None)

COLLECTION_NAME = "qids_nvidia"
ASTRA_DB_APPLICATION_TOKEN = "AstraCS:oGpzOHaScZOdZPTrbyXFWjLu:2d7bd36927434be3ec1a207db19c1a0a2d55e7ec61ed40e02aeaa4036959c669"
ASTRA_DB_API_ENDPOINT = "https://63f72536-7337-4d82-a8a6-cc1f0397c737-us-east-2.apps.astra.datastax.com"
ASTRA_DB_KEYSPACE = "default_keyspace"

client = DataAPIClient(ASTRA_DB_APPLICATION_TOKEN)
database0 = client.get_database(ASTRA_DB_API_ENDPOINT)
wikiDataCollection = database0.get_collection(COLLECTION_NAME)
for chunk in wikiDataCollection.find({"metadata.QID": "Q4109713"}, limit=10):
    print(chunk['metadata']['ChunkID'])

1


In [51]:
from datetime import datetime
import hashlib  # Import the hashlib library
import json

#OMIITED

current_date = datetime.now().date().isoformat()

#OMITTED

def clean_claims(claims):
    cleaned_claims = {}
    for pid,value in claims.items():
        cleaned_claims[pid] = [clean_item(item) for item in value]
    return cleaned_claims
def clean_item(item):
    if 'datavalue' not in item['mainsnak']:
        return {'type': item['mainsnak']['snaktype']}
    if type(item['mainsnak']['datavalue']['value']) is dict:
        value = {'type': item['mainsnak']['datavalue']['type'], **item['mainsnak']['datavalue']['value']}
        if 'entity-type' in value:
            del value['entity-type']
        return value
    return {'type': item['mainsnak']['datavalue']['type'], 'value': item['mainsnak']['datavalue']['value']}

with tqdm(total=9203786) as progressbar:
    with Session() as session:
        entities = session.query(WikidataEntity).join(WikidataID, WikidataEntity.id == WikidataID.id).filter(WikidataID.in_wikipedia == True).offset(OFFSET).yield_per(BATCH_SIZE)
        progressbar.update(OFFSET)
        doc_batch = []
        ids_batch = []

        for entity in entities:
            progressbar.update(1)
            ##if SAMPLE and (entity.id in sample_ids['QID'].values):
            chunks = textifier.chunk_text(entity, tokenizer, max_length=512)
            for chunk_i in range(len(chunks)):
                # Processing Claims
                # print(json.dumps(entity.claims, indent=4))
                md5_hash = hashlib.md5(chunks[chunk_i].encode('utf-8')).hexdigest()
                doc = Document(page_content=chunks[chunk_i],
                                metadata={ "MD5": md5_hash,
                                            "Claims": clean_claims(entity.claims),
                                            "Label": entity.label,
                                            "Description": entity.description,
                                            "Aliases": entity.aliases,
                                            "Date": current_date,
                                            "QID": entity.id,
                                            "ChunkID": chunk_i+1,
                                            "Language": "en",
                                            "Dump Date": "09/18/2024"})
                doc_batch.append(doc)
                ids_batch.append(f"{entity.id}_{chunk_i+1}")

  0%|          | 20/9203786 [00:00<27:29:56, 92.97it/s]

{'P625': [{'type': 'globecoordinate', 'latitude': 67.8823, 'longitude': 12.9841, 'altitude': None, 'precision': 0.0001, 'globe': 'http://www.wikidata.org/entity/Q2'}], 'P18': [{'type': 'string', 'value': 'Sztokfisz muzeum A.jpg'}], 'P856': [{'type': 'string', 'value': 'http://www.datadesign.ws/stockfish.htm'}], 'P17': [{'type': 'wikibase-entityid', 'id': 'Q20'}], 'P31': [{'type': 'wikibase-entityid', 'id': 'Q33506'}], 'P131': [{'type': 'wikibase-entityid', 'id': 'Q487636'}], 'P706': [{'type': 'wikibase-entityid', 'id': 'Q1420953'}], 'P276': [{'type': 'wikibase-entityid', 'id': 'Q258490'}], 'P646': [{'type': 'string', 'value': '/m/0fm4kc'}], 'P373': [{'type': 'string', 'value': 'Tørrfiskmuseum (Å i Lofoten)'}], 'P11693': [{'type': 'string', 'value': '889028492'}]}
{'P131': [{'type': 'wikibase-entityid', 'id': 'Q50157'}], 'P856': [{'type': 'string', 'value': 'http://www.muvi.org/museodiocesanosulmona'}], 'P17': [{'type': 'wikibase-entityid', 'id': 'Q38'}], 'P1949': [{'type': 'string', 'v

  0%|          | 51/9203786 [00:00<19:32:39, 130.81it/s]

{'P31': [{'type': 'wikibase-entityid', 'id': 'Q482994'}], 'P1729': [{'type': 'string', 'value': 'mw0000532941'}], 'P175': [{'type': 'wikibase-entityid', 'id': 'Q26869'}], 'P264': [{'type': 'wikibase-entityid', 'id': 'Q202440'}], 'P136': [{'type': 'wikibase-entityid', 'id': 'Q131272'}, {'type': 'wikibase-entityid', 'id': 'Q1546028'}], 'P577': [{'type': 'time', 'time': '+1975-01-01T00:00:00Z', 'timezone': 0, 'before': 0, 'after': 0, 'precision': 9, 'calendarmodel': 'http://www.wikidata.org/entity/Q1985727'}], 'P5749': [{'type': 'string', 'value': 'B00122DEFI'}], 'P2205': [{'type': 'string', 'value': '5oHdefaxMEZTHRajNjeC87'}], 'P437': [{'type': 'wikibase-entityid', 'id': 'Q15982450'}], 'P1954': [{'type': 'string', 'value': '52503'}]}
{'P31': [{'type': 'wikibase-entityid', 'id': 'Q15416'}], 'P449': [{'type': 'wikibase-entityid', 'id': 'Q5014620'}], 'P646': [{'type': 'string', 'value': '/m/0281hb8'}], 'P495': [{'type': 'wikibase-entityid', 'id': 'Q865'}], 'P2437': [{'type': 'quantity', 'am

  0%|          | 78/9203786 [00:00<14:21:22, 178.08it/s]

...

  0%|          | 384/9203786 [00:03<26:34:39, 96.19it/s]

{'P345': [{'type': 'string', 'value': 'tt0009435'}], 'P161': [{'type': 'wikibase-entityid', 'id': 'Q104340'}], 'P162': [{'type': 'wikibase-entityid', 'id': 'Q72792'}], 'P577': [{'type': 'time', 'time': '+1918-01-01T00:00:00Z', 'timezone': 0, 'before': 0, 'after': 0, 'precision': 9, 'calendarmodel': 'http://www.wikidata.org/entity/Q1985727'}], 'P495': [{'type': 'wikibase-entityid', 'id': 'Q30'}], 'P364': [{'type': 'novalue'}], 'P57': [{'type': 'wikibase-entityid', 'id': 'Q72792'}], 'P462': [{'type': 'wikibase-entityid', 'id': 'Q838368'}], 'P1476': [{'type': 'monolingualtext', 'text': 'Nothing but Trouble', 'language': 'en'}], 'P2603': [{'type': 'string', 'value': '200792'}], 'P750': [{'type': 'wikibase-entityid', 'id': 'Q3368737'}], 'P136': [{'type': 'wikibase-entityid', 'id': 'Q226730'}], 'P31': [{'type': 'wikibase-entityid', 'id': 'Q24862'}], 'P6216': [{'type': 'wikibase-entityid', 'id': 'Q19652'}], 'P646': [{'type': 'string', 'value': '/m/03d0w0s'}], 'P6127': [{'type': 'string', 'val

  0%|          | 398/9203786 [00:03<25:39:52, 99.61it/s] 

{'P175': [{'type': 'wikibase-entityid', 'id': 'Q47122'}], 'P264': [{'type': 'wikibase-entityid', 'id': 'Q427326'}], 'P31': [{'type': 'wikibase-entityid', 'id': 'Q134556'}], 'P155': [{'type': 'wikibase-entityid', 'id': 'Q629135'}], 'P156': [{'type': 'wikibase-entityid', 'id': 'Q2467593'}], 'P2624': [{'type': 'string', 'value': 'nunca-te-olvidare-lyrics-enrique-iglesias'}], 'P361': [{'type': 'wikibase-entityid', 'id': 'Q1935234'}], 'P577': [{'type': 'time', 'time': '+1998-01-00T00:00:00Z', 'timezone': 0, 'before': 0, 'after': 0, 'precision': 10, 'calendarmodel': 'http://www.wikidata.org/entity/Q1985727'}], 'P1889': [{'type': 'wikibase-entityid', 'id': 'Q991337'}], 'P2581': [{'type': 'string', 'value': '00772241n'}], 'P136': [{'type': 'wikibase-entityid', 'id': 'Q547137'}], 'P407': [{'type': 'wikibase-entityid', 'id': 'Q1321'}], 'P1651': [{'type': 'string', 'value': 'pRrjt4htXlE'}], 'P646': [{'type': 'string', 'value': '/m/03mcnb7'}]}
{'P175': [{'type': 'wikibase-entityid', 'id': 'Q399331

KeyboardInterrupt: 

In [54]:
datetime.now().isoformat()

'2024-11-13T15:05:10.103390'

In [ ]:
import sys
sys.path.append('../src')

from wikidataDB import Session, WikidataID, WikidataEntity
from wikidataEmbed import WikidataTextifier
from wikidataRetriever import AstraDBConnect

import json
from tqdm import tqdm
import os
import pickle
from datetime import datetime
import hashlib

SAMPLE = True
BATCH_SIZE = 100
OFFSET = 0
LANGUAGE = 'en'
DUMPDATE = "09/18/2024"
datastax_token = json.load(open("../API_tokens/datastax_wikidata_nvidia.json"))
COLLECTION_NAME = 'wikidata_test_v2'

textifier = WikidataTextifier(with_claim_aliases=False, with_property_aliases=False)
graph_store = AstraDBConnect(datastax_token, COLLECTION_NAME, model='nvidia', batch_size=BATCH_SIZE)

# Load the Sample IDs
sample_ids = None
if SAMPLE:
    sample_ids = pickle.load(open("../data/Evaluation Data/Sample IDs (EN).pkl", "rb"))
    sample_ids = sample_ids[sample_ids['In Wikipedia']]
    sample_ids = sample_ids[sample_ids['Sample 2']]

with tqdm(total=len(sample_ids)) as progressbar:
    for qid in range(0, len(sample_ids), 1000):
        qid_list = sample_ids.iloc[qid:qid+1000]['QID'].tolist()
        with Session() as session:
            entities = session.query(WikidataEntity).filter(WikidataEntity.id.in_(qid_list)).all()

        for entity in entities:
            progressbar.update(1)
            chunks = textifier.chunk_text(entity, graph_store.tokenizer, max_length=graph_store.max_token_size)
            for chunk_i in range(len(chunks)):
                metadata={
                    "QID": entity.id,
                    "ChunkID": chunk_i+1
                }
                graph_store.add_document(id=f"{entity.id}_{chunk_i+1}", text=chunks[chunk_i], metadata=metadata)
    graph_store.push_batch()

 79%|███████▉  | 870/1100 [00:23<00:03, 61.69it/s]

Server disconnected without sending a response.


100%|██████████| 1100/1100 [00:31<00:00, 34.50it/s]
